# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [ ]:
# Imports here
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import numpy as np


## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [ ]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [ ]:
# TODO: Define your transforms for the training, validation, and testing sets
data_transforms = transforms.Compose([
    transforms.Resize(256),                  # Resize the image to 256x256 pixels
    transforms.CenterCrop(224),              # Crop the image to 224x224 pixels (size expected by most models)
    transforms.ToTensor(),                   # Convert the image to a PyTorch Tensor (with values between 0 and 1)
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Normalize using ImageNet mean
                         std=[0.229, 0.224, 0.225])  # Normalize using ImageNet std
])
                               
data_dir = 'Cat_Dog_data'

# TODO: Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.Resize(256),                  # Resize the image to 256x256 pixels
                                    transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])


valid_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
test_dataset = datasets.ImageFolder(test_dir, transform=test_transforms)
valid_dataset = datasets.ImageFolder(valid_dir, transform=valid_transforms)


trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=64)
validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=64)
# TODO: Load the datasets with ImageFolder
# image_datasets = datasets.ImageFolder(data_dir, transform=data_transforms)
# # TODO: Using the image datasets and the trainforms, define the dataloaders
# dataloader = torch.utils.data.DataLoader(image_datasets, batch_size=32, shuffle=True)


### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [ ]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [ ]:
model = models.vgg16(pretrained=True)
model

In [ ]:

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the pre-trained VGG16 model
model = models.vgg16(pretrained=True)

# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False
    
    
in_features_of_pretrained_model = model.classifier[0].in_features

# Define the classifier
classifier = nn.Sequential(
    nn.Linear(in_features_of_pretrained_model, 4096),
    nn.ReLU(),
    nn.Dropout(p=0.2),
    nn.Linear(4096, 102),  # Adjust output size based on your dataset
    nn.LogSoftmax(dim=1)
)

# Assign the classifier to the model
model.classifier = classifier
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)

# Move the model to the GPU (if available)
model = model.to(device)


# Function to calculate accuracy
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, 1)
    return torch.sum(preds == labels).item()



In [ ]:
model

In [ ]:
# TODO: Build and train your network
epochs = 2
print_every = 3

# for each epoch
for e in range(epochs):
    step = 0
    running_train_loss = 0
    running_valid_loss = 0

    # Training phase
    for images, labels in trainloader:
        step += 1

        # turn model to train mode
        model.train()

        # move images and labels to device
        images, labels = images.to(device), labels.to(device)

        # zero gradients
        optimizer.zero_grad()

        # forward pass
        outputs = model(images)

        # calculate loss
        train_loss = criterion(outputs, labels)

        # backward pass
        train_loss.backward()

        # optimizer step
        optimizer.step()

        running_train_loss += train_loss.item()

        # Print training progress
        if step % print_every == 0 :
            print(f"Epoch: {e+1}/{epochs}, Batch Completion: {(step / len(trainloader)) * 100:.2f}%")

    # Validation phase
    model.eval()
    running_accuracy = 0
    running_valid_loss = 0

    with torch.no_grad():
        for images, labels in validloader:

            # Move images and labels to device
            images, labels = images.to(device), labels.to(device)

            # forward pass
            outputs = model(images)

            # calculate validation loss
            valid_loss = criterion(outputs, labels)
            running_valid_loss += valid_loss.item()

            # calculate accuracy
            _, top_class = outputs.max(1)  # No need to exponentiate
            equals = top_class == labels
            running_accuracy += torch.mean(equals.float()).item()

    # Print statistics for the epoch
    average_train_loss = running_train_loss / len(trainloader)
    average_valid_loss = running_valid_loss / len(validloader)
    accuracy = running_accuracy / len(validloader)

    print(f"Train Loss: {average_train_loss:.3f}")
    print(f"Valid Loss: {average_valid_loss:.3f}")
   


## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
# TODO: Do validation on the test set
# TODO: Do validation on the test set
model.eval()  # Set model to evaluation mode
test_loss = 0.0
test_acc = 0.0

with torch.no_grad():  # No gradient computation for testing
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move test data to GPU
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * inputs.size(0)  # Accumulate test loss
        test_acc += accuracy(outputs, labels)  # Accumulate test accuracy

# Compute average test loss and accuracy
test_loss /= len(testloader.dataset)
test_acc /= len(testloader.dataset)

# Print test results
print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
# TODO: Save the checkpoint 

# TODO: Save the checkpoint 
# Attach the class to index mapping to the model
model.class_to_idx = train_dataset.class_to_idx

# Define a checkpoint dictionary
checkpoint = {
    'state_dict': model.state_dict(),  # Save model state
    'class_to_idx': model.class_to_idx,  # Save class to index mapping
    'classifier': model.classifier,  # Save classifier architecture
    'optimizer_state_dict': optimizer.state_dict(),  # Save optimizer state
    'epochs': epochs  # Save number of epochs
}

# Save the checkpoint
torch.save(checkpoint, 'checkpoint.pth')

print("Model checkpoint saved.")


## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
# TODO: Write a function that loads a checkpoint and rebuilds the model

def load_checkpoint(filepath):
    """
    Load a model checkpoint and rebuild the model.
    
    Parameters:
    filepath (str): Path to the checkpoint file.
    
    Returns:
    model (torch.nn.Module): Rebuilt model with loaded weights and classifier.
    optimizer (torch.optim.Optimizer): Optimizer loaded with the state.
    """
    # Load the saved checkpoint
    checkpoint = torch.load(filepath, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
    
    # Load the pre-trained model architecture
    model = models.vgg19_bn(pretrained=True)
    
    # Rebuild the classifier from the checkpoint
    model.classifier = checkpoint['classifier']
    
    # Load the state dictionary (weights)
    model.load_state_dict(checkpoint['state_dict'])
    
    # Load class-to-index mapping
    model.class_to_idx = checkpoint['class_to_idx']
    
    # Create the optimizer and load its state
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    # Return the rebuilt model and optimizer
    return model, optimizer

# Example usage:
loaded_model, loaded_optimizer = load_checkpoint('checkpoint.pth')
print("Model and optimizer loaded successfully.")


# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    pil_image = Image.open(image_path)

    # Resize where the shortest side is 256 pixels
    aspect_ratio = pil_image.width / pil_image.height
    if pil_image.width < pil_image.height:
        pil_image = pil_image.resize((256, int(256 / aspect_ratio)))
    else:
        pil_image = pil_image.resize((int(256 * aspect_ratio), 256))
    
    # Center crop to 224x224
    left = (pil_image.width - 224) / 2
    top = (pil_image.height - 224) / 2
    right = left + 224
    bottom = top + 224
    pil_image = pil_image.crop((left, top, right, bottom))

    # Convert to numpy array
    np_image = np.array(pil_image) / 255.0  # Scale pixel values from [0, 255] to [0, 1]

    # Normalize the image with the means and std deviations of ImageNet
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    np_image = (np_image - mean) / std

    # Reorder color channel to be the first dimension (PyTorch expects this format)
    np_image = np_image.transpose((2, 0, 1))
    tensor = torch.from_numpy(np_image)
    tensor = tensor.type(torch.FloatTensor)
   

    return tensor

    
    # TODO: Process a PIL image for use in a PyTorch model

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5, device="cuda"):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    image = process_image(image_path)
    image = image.unsqueeze(0)

    # move to device
    image = image.to(device)
    model.eval()
    with torch.no_grad():
        ps = torch.exp(model(image))
        
    idx_to_flower = {v:cat_to_name[k] for k, v in model.class_to_idx.items()}    
    
    top_ps, top_classes = ps.topk(topk, dim=1)
    predicted_flowers = [idx_to_flower[i] for i in top_classes.tolist()[0]]

    return top_ps.tolist()[0], predicted_flowers
    

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes
# TODO: Display an image along with the top 5 classes
import json
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# Load class to name mapping
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Function to show image
def imshow(image, ax=None, title=None):
    """Display a PyTorch tensor as an image."""
    if ax is None:
        fig, ax = plt.subplots()
    # Convert the tensor back to PIL Image format
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo normalization
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Clip to ensure the values are between 0 and 1
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    if title:
        ax.set_title(title)
    ax.axis('off')
    
    return ax

def plot_sanity_check(image_path, model, cat_to_name, topk=5):
    """Display an image along with the top K predicted classes and probabilities."""
    # Predict top K classes
    probs, classes = predict(image_path, model, topk)
    
    # Convert class indices to actual names using cat_to_name mapping
    class_names = [cat_to_name[str(cls)] for cls in classes]
    
    # Plot the input image
    plt.figure(figsize=(6, 10))
    ax = plt.subplot(2, 1, 1)
    
    # Process the image and display it
    image = process_image(image_path)
    image = torch.from_numpy(image)
    imshow(image, ax=ax, title=class_names[0])
    
    # Plot the top K probabilities as a bar graph
    plt.subplot(2, 1, 2)
    y_pos = np.arange(len(class_names))
    plt.barh(y_pos, probs)
    plt.yticks(y_pos, class_names)
    plt.gca().invert_yaxis()  # Invert to have the top class on top
    
    plt.show()


